# Importe de librerias

In [69]:
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from nltk.corpus import stopwords
import collections
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.metrics import make_scorer


#descargas
#nltk.download('stopwords')
#nltk.download('punkt')

# Ampliacion del dataset

In [3]:
ds_descrip = pd.read_csv('properati_argentina_2021_decrip.csv')

In [4]:
ds_descrip.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...


In [5]:
ds_descrip.dtypes

id                      object
property_description    object
dtype: object

In [6]:
ds_descrip.shape

(460154, 2)

Realizo transformaciones sobre el dataset


In [7]:
descriptions = ds_descrip['property_description'].astype(str)
descriptions = descriptions.str.replace('[{}]'.format(string.punctuation), '')

C:\Users\EQUIPO~1\AppData\Local\Temp/ipykernel_10292/642431205.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  descriptions = descriptions.str.replace('[{}]'.format(string.punctuation), '')


In [8]:
descriptions

0         Corredor Responsable MARIA ALEJANDRA GENOVEVA ...
1         ¡HERMOSO PH TIPO CASA DE 2 AMBIENTESbrAPTO CRÉ...
2         Excelente PH en Núñez 4 amb Oportunidad  Total...
3         UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO ...
4         Ph 2 Amb Villa Crespo exclusivo uso profesiona...
                                ...                        
460149    Corredor Responsable Hector Garaygorta  Cristi...
460150    Corredor Responsable MARIA DEL ROSARIO NEME  1...
460151    En cumplimiento de las leyes provinciales vige...
460152    En cumplimiento de las leyes provinciales vige...
460153    CASA EN VENTA EN BARILOCHE – Lomas del Cauquén...
Name: property_description, Length: 460154, dtype: object

### Combino dataset de descripciones con dataset popiedades obtenidos TP1

A traves del ID de cada descripcion puedo saber de que propiedad es la descipcion de cada fila

In [9]:
ds_train= pd.read_csv('properati_argentina_train.csv')
ds_test = pd.read_csv('properati_argentina_test.csv')

In [10]:
ds_train.head()

,Unnamed: 0,id,latitud,longitud,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,precioxm2_relativo_q
0,0,CLj2GNCsBreAzTKFXCaLIQ==,-34.601807,-58.469965,Departamento,2.0,1.0,53.0,49.0,80000.0,bajo
1,1,TvqKIPzzT1urYfVhh8tC7w==,-34.580088,-58.424965,Departamento,2.0,1.0,35.0,35.0,79900.0,medio
2,2,QRs3RUjw1RS5/WRne8ApbA==,-34.551481,-58.452358,Departamento,1.0,1.0,25.0,20.0,69000.0,medio
3,3,ju90QBsV36i7DCOOWCJpaA==,-34.632290,-58.464844,Departamento,5.0,3.0,75.0,75.0,150000.0,medio
4,4,uxD4SK3jtGbmxfIlN03UMQ==,-34.549279,-58.463074,Departamento,2.0,1.0,46.0,34.0,85000.0,bajo


In [11]:
ds_test.head()

,Unnamed: 0,id,latitud,longitud,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,precioxm2_relativo_q
0,0,1MIZ1gFNRkTXELdcBl+opQ==,-34.598679,-58.499717,Departamento,1.0,1.0,38.0,28.0,85000.0,medio
1,1,hi5vMb1IpzsBoUGGVGPCXA==,-34.574987,-58.420611,Departamento,2.0,1.0,150.0,70.0,520000.0,alto
2,2,f57uQb0jzaTRg98/B71HOg==,-34.586527,-58.398279,Departamento,2.0,1.0,43.0,41.0,87000.0,medio
3,3,gsO2peJbSoSQ0hI1E5TH8g==,-34.577647,-58.406064,Departamento,4.0,3.0,130.0,118.0,690000.0,alto
4,4,4cnMe44uQLFiAnBC1SXjUg==,-34.560998,-58.464248,Departamento,1.0,1.0,33.0,29.0,95000.0,alto


In [12]:
#Eliminamos columna Unnamed

ds_train.drop(columns="Unnamed: 0", inplace=True)
ds_test.drop(columns="Unnamed: 0", inplace=True)

In [13]:
#Dataset train
ds_train = ds_train.join(ds_descrip.set_index('id'), on='id')
ds_train.head()

#Dataset test
ds_test = ds_test.join(ds_descrip.set_index('id'), on='id')

## Seleccion de atributos de propiedades

- Seleccionamos atributos que podrian llegar a aportar informacion importante al dataset
- A traves de regex buscamos palabras claves en las descripciones de las propiedades
- Colocamos un "1" si la propiedad posee esa palabra, de lo contrario "0"

### Pileta

In [14]:
regexp = re.compile(r'piscina|pileta/gi', re.IGNORECASE)

ds_train['piscina'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['piscina'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen pileta

In [15]:
ds_train["piscina"].value_counts()

0    60313
1     7699
Name: piscina, dtype: int64

### Balcon 

In [16]:
regexp = re.compile(r'balcon', re.IGNORECASE)

ds_train['balcon'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['balcon'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen balcon

In [17]:
ds_train["balcon"].value_counts()

0    55956
1    12056
Name: balcon, dtype: int64

### Patio

In [18]:
regexp = re.compile(r'patio', re.IGNORECASE)

ds_train['patio'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['patio'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen patio

In [19]:
ds_train["patio"].value_counts()

0    57091
1    10921
Name: patio, dtype: int64

### Terraza

In [20]:
regexp = re.compile(r'terraza', re.IGNORECASE)

ds_train['terraza'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['terraza'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen terraza

In [21]:
ds_train["terraza"].value_counts()

0    47376
1    20636
Name: terraza, dtype: int64

### Luminoso

In [22]:
regexp = re.compile(r'luminosa|luminoso/gi', re.IGNORECASE)

ds_train['luminoso'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['luminoso'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades son caracterizadas como luminosas

In [23]:
ds_train["luminoso"].value_counts()

0    65280
1     2732
Name: luminoso, dtype: int64

### Gimnasio

In [24]:
regexp = re.compile(r'gimnasio', re.IGNORECASE)

ds_train['gimnasio'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['gimnasio'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen gimnasio

In [25]:
ds_train["gimnasio"].value_counts()

0    62685
1     5327
Name: gimnasio, dtype: int64

### Garaje 

In [26]:
regexp = re.compile(r'garage|cochera/gi', re.IGNORECASE)

ds_train['garaje'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['garaje'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [27]:
ds_train["garaje"].value_counts()

0    67398
1      614
Name: garaje, dtype: int64

# Modelos - XGboost

### Ingenieria de caracteristicas

In [28]:
ds_train.head()

,id,latitud,longitud,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,precioxm2_relativo_q,property_description,piscina,balcon,patio,terraza,luminoso,gimnasio,garaje
0,CLj2GNCsBreAzTKFXCaLIQ==,-34.601807,-58.469965,Departamento,2.0,1.0,53.0,49.0,80000.0,bajo,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...,0,1,0,0,0,0,0
1,TvqKIPzzT1urYfVhh8tC7w==,-34.580088,-58.424965,Departamento,2.0,1.0,35.0,35.0,79900.0,medio,"Ubicado en Palermo, en una zona de muchos medi...",0,0,0,0,0,0,0
2,QRs3RUjw1RS5/WRne8ApbA==,-34.551481,-58.452358,Departamento,1.0,1.0,25.0,20.0,69000.0,medio,Corredor Responsable: Diego Enrique Mastrangel...,0,1,0,0,0,0,0
3,ju90QBsV36i7DCOOWCJpaA==,-34.632290,-58.464844,Departamento,5.0,3.0,75.0,75.0,150000.0,medio,EXCELENTE DEPARTAMENTO DE 5 AMBIENTES DE 75 M2...,0,0,0,1,0,0,0
4,uxD4SK3jtGbmxfIlN03UMQ==,-34.549279,-58.463074,Departamento,2.0,1.0,46.0,34.0,85000.0,bajo,Muy buen departamento de 2 ambientes en O'Higg...,0,0,1,0,0,0,0


In [29]:
ds_test.head()

,id,latitud,longitud,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,precioxm2_relativo_q,property_description,piscina,balcon,patio,terraza,luminoso,gimnasio,garaje
0,1MIZ1gFNRkTXELdcBl+opQ==,-34.598679,-58.499717,Departamento,1.0,1.0,38.0,28.0,85000.0,medio,Departamento 38m² 1 ambiente con Gas Natural e...,0,0,0,0,0,0,0
1,hi5vMb1IpzsBoUGGVGPCXA==,-34.574987,-58.420611,Departamento,2.0,1.0,150.0,70.0,520000.0,alto,Depto muy exclusivo!!!! con excelente terraza ...,0,0,1,1,0,0,0
2,f57uQb0jzaTRg98/B71HOg==,-34.586527,-58.398279,Departamento,2.0,1.0,43.0,41.0,87000.0,medio,Gran Oportunidad ! Re retasado!<br>Depto. 2 am...,0,0,0,0,0,0,0
3,gsO2peJbSoSQ0hI1E5TH8g==,-34.577647,-58.406064,Departamento,4.0,3.0,130.0,118.0,690000.0,alto,<br>IMPECABLE UNIDAD DE 4 AMBIENTES CON DEPEN...,0,0,0,0,1,0,0
4,4cnMe44uQLFiAnBC1SXjUg==,-34.560998,-58.464248,Departamento,1.0,1.0,33.0,29.0,95000.0,alto,DEPARTAMENTO EN ESQUINA CON AMENITIES <br><br...,0,0,0,0,0,0,0


Eliminos columnas que no voy a utilizar en el modelo
- id, cumplio su funcion en el punto anterioir, no aporta nada en el modelo.
- precioxm2_relativo_q, esta relacionado con el precio
- property_description por la misma razon de id

In [30]:
columnas_a_eliminar = ["id", "precioxm2_relativo_q", "property_description"]

ds_train.drop(columns=columnas_a_eliminar, inplace=True)
ds_test.drop(columns=columnas_a_eliminar, inplace=True)

Normalizo variables numericas

In [31]:
variables_a_normalzar = ["latitud", "longitud", "property_rooms", "property_bedrooms", "property_surface_covered", "property_surface_total"]
train_cuantitativas = ds_train.loc[:, variables_a_normalzar]
test_cuantitativas =  ds_test.loc[:, variables_a_normalzar]

Entreno el scaler

In [32]:
scaler = StandardScaler()
scaler.fit(train_cuantitativas)

StandardScaler()

In [33]:
ds_train[variables_a_normalzar] = scaler.transform(train_cuantitativas)
ds_test[variables_a_normalzar] = scaler.transform(test_cuantitativas)

In [34]:
ds_train.head()

,latitud,longitud,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,piscina,balcon,patio,terraza,luminoso,gimnasio,garaje
0,-0.226971,-0.843338,Departamento,-0.572024,-0.901181,-0.360814,-0.422360,80000.0,0,1,0,0,0,0,0
1,0.613885,0.359192,Departamento,-0.572024,-0.901181,-0.611332,-0.761908,79900.0,0,0,0,0,0,0,0
2,1.721401,-0.372818,Departamento,-1.448310,-0.901181,-0.750509,-1.125710,69000.0,0,1,0,0,0,0,0
3,-1.407153,-0.706479,Departamento,2.056835,1.409468,-0.054625,0.208230,150000.0,0,0,0,1,0,0,0
4,1.806656,-0.659172,Departamento,-0.572024,-0.901181,-0.458238,-0.786162,85000.0,0,0,1,0,0,0,0


In [35]:
ds_test.head()

,latitud,longitud,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,piscina,balcon,patio,terraza,luminoso,gimnasio,garaje
0,-0.105881,-1.638376,Departamento,-1.448310,-0.901181,-0.569579,-0.931682,85000.0,0,0,0,0,0,0,0
1,0.811376,0.475542,Departamento,-0.572024,-0.901181,0.989200,0.086963,520000.0,0,0,1,1,0,0,0
2,0.364577,1.072312,Departamento,-0.572024,-0.901181,-0.499991,-0.616388,87000.0,0,0,0,0,0,0,0
3,0.708381,0.864258,Departamento,1.180548,1.409468,0.710847,1.251129,690000.0,0,0,0,0,1,0,0
4,1.352950,-0.690547,Departamento,-1.448310,-0.901181,-0.639168,-0.907429,95000.0,0,0,0,0,0,0,0


Aplico dummies a la variable property_type

In [36]:
ds_train = pd.get_dummies(ds_train, columns=["property_type"], drop_first=True)
ds_test = pd.get_dummies(ds_test, columns=["property_type"], drop_first=True)

### Creamos modelo con mismos hiperparametros TP1

In [37]:
features = list(ds_train.columns.values)
features.pop(features.index("property_price"))

target = ["property_price"]

#train
x_train = ds_train[features]
y_train = ds_train[target]

#test
x_test = ds_test[features]
y_test = ds_test[target]

In [53]:
#mejores parametros obtenidos en TP1
mejores_param_tp1 = {'random_state': 4, 'min_child_weight': 17, 'max_depth': 3, 'gamma': 31, 'criterion': 'entropy', 'colsample_bytree': 0.775, 'ccp_alpha': 0.42857142857142855}

#creo el modelo
xgb_model = xgb.XGBRegressor(random_state = 4, min_child_weight = 17,  max_depth= 3, gamma= 31, criterion= "entropy" , colsample_bytree= 0.775, ccp_alpha = 0.42857142857142855)

Enterno el modelo con dataset extendido

In [54]:
xgb_model.fit(x_train, y_train)

[11:38:27] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.



XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             ccp_alpha=0.42857142857142855, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.775, criterion='entropy',
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=31, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=3, max_leaves=0, min_child_weight=17, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, ...)

Obtenemos metricas con datos de entrenamiento

In [62]:
y_pred = xgb_model.predict(x_train)

mse = mean_squared_error(y_train, y_pred, squared=True)
rmse = mean_squared_error(y_train, y_pred, squared=False)
r2 = r2_score(y_train, y_pred)

print("MSE en datos de entrenamiento: " + str(mse))
print("RMSE en datos de entrenamiento: " + str(rmse))
print("R2 score en datos de entrenamiento: " + str(r2))

MSE en datos de entrenamiento: 2714071692.9594398
RMSE en datos de entrenamiento: 52096.753190188734
R2 score en datos de entrenamiento: 0.876869714024928


Obtenemos las metricas con los datos de testeo

In [63]:
y_pred = xgb_model.predict(x_test)

mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("MSE en datos de test: " + str(mse))
print("RMSE en datos de test: " + str(rmse))
print("R2 score en datos de test: " + str(r2))

MSE en datos de test: 2653244989.0927453
RMSE en datos de test: 51509.659182455725
R2 score en datos de test: 0.870703979170422


In [64]:
#Atributos considerados y su importancia
sorted(list(zip(x_train.columns.to_list(), xgb_model.feature_importances_)), key=lambda x: -x[1])

[('property_surface_covered', 0.36584526),
 ('property_type_Departamento', 0.25237903),
 ('property_surface_total', 0.09806521),
 ('piscina', 0.0734702),
 ('gimnasio', 0.0700343),
 ('latitud', 0.04076399),
 ('longitud', 0.036223475),
 ('property_rooms', 0.017149841),
 ('property_type_PH', 0.013147939),
 ('patio', 0.012245812),
 ('property_bedrooms', 0.0075936834),
 ('terraza', 0.0053010047),
 ('garaje', 0.0038436756),
 ('balcon', 0.0033944524),
 ('luminoso', 0.00054218795)]

### Creamos modelo con nuevos parametros

#### Optimizacion de parametros con Random Search

#explicacion

In [70]:
#KFOLD CV Random Search para buscar el mejor arbol (los mejores atributos, hiperparametros,etc)

#Cantidad de combinaciones que quiero porbar
n=10

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],                  #Luego de probar con varias combinaciones de parametros se
               'ccp_alpha':np.linspace(0,0.5,15),               #llegó a la conclusión de que Random Search encuentra los
               'max_depth':list(range(3,4)),                    #hiperparametros más adecuados con esta grilla 
               'random_state':list(range(0,6)),                 #de parametros
               'gamma':list(range(31,32)),
               'min_child_weight':list(range(17,18)),
               'colsample_bytree':np.linspace(0.75,0.8,15)}
                
#Cantidad de splits para el Cross Validation
folds=8

#Regresor
xgb_model_rd_search = xgb.XGBRegressor()

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=xgb_model_rd_search,
                              param_distributions = params_grid,
                              scoring=scorer_fn,
                              n_iter=n, cv=folds, random_state=5) 


#Busco los hiperparamtros que optimizan MSE
randomcv.fit(x_train,y_train)

[12:00:35] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[12:00:36] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[12:00:37] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[12:00:39] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.

[12:00:40] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:76

RandomizedSearchCV(cv=8,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
       0.35714286, 0.39285714, 0.42857143, 0.46428571, 0.5       ]),
                                      

In [71]:
#Mejores hiperparametros
print(randomcv.best_params_)

#Mejor métrica
mse = randomcv.best_score_

print("MSE en datos de entrnamiento: " + str((mse)))
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))


{'random_state': 5, 'min_child_weight': 17, 'max_depth': 3, 'gamma': 31, 'criterion': 'entropy', 'colsample_bytree': 0.7821428571428571, 'ccp_alpha': 0.46428571428571425}
MSE en datos de entrnamiento: 2999983889.396943
RMSE en datos de entrnamiento: 54772.10868130734


Obtenemos el mejor modelo y lo entrenamos

In [72]:
xgb_model = xgb.XGBRegressor().set_params(**randomcv.best_params_)
xgb_model.fit(x_train, y_train)

[12:08:48] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "ccp_alpha", "criterion" } are not used.



XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             ccp_alpha=0.46428571428571425, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7821428571428571,
             criterion='entropy', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=31, gpu_id=-1, grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=3, max_leaves=0, min_child_weight=17, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, ...)

Verificamos su performance en los datos de entrenamiento

In [73]:
y_pred = xgb_model.predict(x_train)

mse = mean_squared_error(y_train, y_pred, squared=True)
rmse = mean_squared_error(y_train, y_pred, squared=False)
r2 = r2_score(y_train, y_pred)

print("MSE en datos de entrenamiento: " + str(mse))
print("RMSE en datos de entrenamiento: " + str(rmse))
print("R2 score en datos de entrenamiento: " + str(r2))

MSE en datos de entrenamiento: 2753686535.762362
RMSE en datos de entrenamiento: 52475.58037566009
R2 score en datos de entrenamiento: 0.8750724929213606


Verificamos su performance en los datos de test

In [74]:
y_pred = xgb_model.predict(x_test)

mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("MSE en datos de test: " + str(mse))
print("RMSE en datos de test: " + str(rmse))
print("R2 score en datos de testeo: " + str(r2))

MSE en datos de test: 2667839240.6101675
RMSE en datos de test: 51651.13010002944
R2 score en datos de testeo: 0.8699927826333718
